In [ ]:
!nvidia-smi

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Check the dataset
from datasets import load_dataset
ds = load_dataset("hdparmar/irish-traditional-tunes", split="train")
sample = ds[0]
display(sample["image"])
print(sample["text"])

In [ ]:
from huggingface_hub import hf_hub_download
ckpt_path = hf_hub_download(repo_id="riffusion/riffusion-model-v1", filename="riffusion-model-v1.ckpt")

In [ ]:
# 2xA6000:
BATCH_SIZE = 4
N_GPUS = 2
ACCUMULATE_BATCHES = 2
GPU_LIST = ",".join((str(x) for x in range(N_GPUS))) + ","
print(f"Using GPUs: {GPU_LIST}")

import os
os.environ["BATCH_SIZE"] = str(BATCH_SIZE) 
os.environ["N_GPUS"] = str(N_GPUS) 
os.environ["ACCUMULATE_BATCHES"] = str(ACCUMULATE_BATCHES)
os.environ["GPU_LIST"] = GPU_LIST
os.environ["CKPT_PATH"] = ckpt_path

!echo "$BATCH_SIZE"
!echo "$N_GPUS"
!echo "$ACCUMULATE_BATCHES"
!echo "$GPU_LIST"
!echo "$CKPT_PATH"

In [ ]:
# Run training
!(python main.py \
    -t \
    --base configs/stable-diffusion/pokemon.yaml \
    --gpus "$GPU_LIST" \
    --scale_lr False \
    --num_nodes 1 \
    --check_val_every_n_epoch 6 \
    --finetune_from "$CKPT_PATH" \
    data.params.batch_size="$BATCH_SIZE" \
    lightning.trainer.accumulate_grad_batches="$ACCUMULATE_BATCHES" \
    data.params.validation.params.n_gpus="$N_GPUS" \
)